In [1]:
import pandas as pd
import helpers
import json

KeyboardInterrupt: 

In [ ]:
train_df = pd.read_json('./data/train_dataset.jl', lines=True, chunksize=500)
test_df = pd.read_json('./data/test_dataset.jl', lines=True, chunksize=500)
items = helpers.load_items()

## Interactions

In [ ]:
def process(df):
    def parse_row(x, events, drop_target):
        raw_events = x['user_history']
        events.extend([
            (x.name, event['event_info'], event['event_type'], event['event_timestamp'], x['item_bought'] if drop_target else None) for event in raw_events 
        ])
    # 'user_id', 'item_id', 'event_type', 'event_timestamp', 'target'
    interactions_df = pd.DataFrame()
    drop_target = False
    for raw_data in df:
        drop_target = 'item_bought' in raw_data.columns
        events = []
        raw_data.apply(lambda x: parse_row(x, events, drop_target), axis=1)
        interactions_df = pd.concat([interactions_df, pd.DataFrame(events)], copy=False)

    interactions_df.columns = ['user_id', 'item_id', 'event_type', 'event_timestamp', 'target']
    if drop_target:
        interactions_df = interactions_df.drop(columns=['target'])
    return interactions_df
    

In [ ]:
interactions_df = process(train_df)

In [ ]:
interactions_df[interactions_df['user_id'] == 932]

In [45]:
items[1361154]

{'title': 'Relogio Bluetooth Wifi Smartwatch A1 Camera Celular Chip',
 'domain_id': 'MLB-SMARTWATCHES',
 'product_id': nan,
 'price': 99.0,
 'category_id': 'MLB135384',
 'condition': 'new'}

In [32]:
interactions_df.to_csv('./data/interactions_test.csv', index=False)

## Items

In [8]:
df_data = {
    'item_id': [],
    'title': [],
    'domain_id':[],
    'product_id':[],
    'price':[],
    'category_id':[],
    'condition':[],
}
for key in items.keys():
    df_data['item_id'].append(key)
    df_data['title'].append(items[key]['title'])
    df_data['domain_id'].append(items[key]['domain_id'])
    df_data['product_id'].append(items[key]['product_id'])
    df_data['price'].append(items[key]['price'])
    df_data['category_id'].append(items[key]['category_id'])
    df_data['condition'].append(items[key]['condition'])
    
items_df = pd.DataFrame(df_data)

In [28]:
items_df

,item_id,title,domain_id,product_id,price,category_id,condition
0,111260,Casa Sola En Venta Con Gran Patio Solo Pago De...,MLM-INDIVIDUAL_HOUSES_FOR_SALE,NaN,1150000.00,MLM170527,new
1,871377,Resident Evil Origins Collection Nintendo Swit...,MLM-VIDEO_GAMES,15270800.0,1392.83,MLM151595,new
2,490232,Falda De Imitación Piel Negra,MLM-SKIRTS,NaN,350.00,MLM7697,new
3,1150706,Powercolor Red Devil Radeon Rx 580 8gb Gddr5,MLM-GRAPHICS_CARDS,NaN,3200.00,MLM9761,used
4,934912,Laptop Hp Nx6320 Core Duo Con Puerto Db9 Windo...,MLM-NOTEBOOKS,NaN,1599.00,MLM1652,used
...,...,...,...,...,...,...,...
2102272,1099649,Carrinho De Bebê Stoke,MLB-BABY_STROLLERS,NaN,1600.00,MLB1386,used
2102273,1482187,Grelha Para Hambúrguer Preta Com Cabo Em Madei...,MLB-KITCHEN_SUPPLIES,NaN,69.90,MLB193425,new
2102274,1118904,Meia Tam 7/8 Anti Embolia Trombose Antitrombo,MLB-SOCKS,NaN,118.00,MLB108791,new
2102275,237229,Pano De Boca Cremer Menina Luxo Bordado C/3 Und,MLB-DISPOSABLE_BABY_DIAPERS,NaN,26.90,MLB40629,new


In [9]:
items_df.to_csv('./data/items.csv', index=False)

## User-Item Candidates

In [ ]:
def get_candidates(user):
    df = interactions_df[(interactions_df['user_id'] == user)]# & (interactions['event_type'] != 'search')]
    items_interacted = set(df['item_id'])
    domains = set(items_dict[int(item)]['domain_id'] for item in items_interacted)
    return [str(x) for x in sum([domain_item_dict[d] for d in domains], [])]

In [ ]:
%%time
users = train_users[:5000]
users_column = []
items_column = []
user_lengths = []
for u in users:
    candidates = get_candidates(u)
    items_column += candidates
    users_column += [u] * len(candidates)
    user_lengths.append((u, len(candidates)))
pairs = pd.DataFrame({'user_id': users_column, 'item_id': items_column})
pairs